In [388]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from wordcloud import WordCloud
from scipy import stats
import missingno as msno
import re

In [389]:
df = pd.read_csv('DataAnalyst.csv')

In [390]:
df.shape

(2253, 16)

In [391]:
len(df['Location'])

2253

In [392]:
df['Location'].isnull().sum()

0

In [393]:
df[['city','state','none']] = df['Location'].str.split(', ',expand=True)

In [394]:
df['none'].value_counts()

CO    8
Name: none, dtype: int64

In [395]:
df['state'].replace({'Arapahoe': 'CO'})

0       NY
1       NY
2       NY
3       NY
4       NY
        ..
2248    CO
2249    CO
2250    CO
2251    CO
2252    CO
Name: state, Length: 2253, dtype: object

In [356]:
df.head(7)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply,city,State
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True,New York,NY
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1,New York,NY
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,-1,New York,NY
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),-1,-1,New York,NY
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel\n3.9,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True,New York,NY
5,5,Data Analyst,$37K-$66K (Glassdoor est.),About Cubist\nCubist Systematic Strategies is ...,3.9,Point72\n3.9,"New York, NY","Stamford, CT",1001 to 5000 employees,2014,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,-1,-1,NaN,NaN
6,6,Business/Data Analyst (FP&A),$37K-$66K (Glassdoor est.),Two Sigma is a different kind of investment ma...,4.4,Two Sigma\n4.4,"New York, NY","New York, NY",1001 to 5000 employees,2001,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,-1,-1,NaN,NaN


In [404]:
df=df.replace(-1,np.nan)
df=df.replace(-1.0,np.nan)
df=df.replace('-1',np.nan)
df=df.replace('unknown',np.nan)

In [405]:
df['Job Description'] = df['Job Description'].apply(lambda x: re.sub('\n', ' ', str(x)))
df['Company Name'] = df['Company Name'].apply(lambda x: re.sub('\n...', '', str(x))) #replace all instances of the new line and company rating from 

In [406]:
df[['Min_Salary','Max_Salary']]=df['Salary Estimate'].str.split('-',expand=True)

In [407]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,Industry,Sector,Revenue,Competitors,Easy Apply,city,state,none,Min_Salary,Max_Salary
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice,"New York, NY","New York, NY",201 to 500 employees,1961.0,...,Social Assistance,Non-Profit,$100 to $500 million (USD),NaN,True,New York,NY,None,$37K,$66K (Glassdoor est.)
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview Provides analytical and technical su...,3.8,Visiting Nurse Service of New York,"New York, NY","New York, NY",10000+ employees,1893.0,...,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),NaN,NaN,New York,NY,None,$37K,$66K (Glassdoor est.)
2,2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace,"New York, NY","New York, NY",1001 to 5000 employees,2003.0,...,Internet,Information Technology,Unknown / Non-Applicable,GoDaddy,NaN,New York,NY,None,$37K,$66K (Glassdoor est.)
3,3,Data Analyst,$37K-$66K (Glassdoor est.),Requisition NumberRR-0001939 Remote:Yes We col...,4.1,Celerity,"New York, NY","McLean, VA",201 to 500 employees,2002.0,...,IT Services,Information Technology,$50 to $100 million (USD),NaN,NaN,New York,NY,None,$37K,$66K (Glassdoor est.)
4,4,Reporting Data Analyst,$37K-$66K (Glassdoor est.),ABOUT FANDUEL GROUP FanDuel Group is a world-...,3.9,FanDuel,"New York, NY","New York, NY",501 to 1000 employees,2009.0,...,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD),DraftKings,True,New York,NY,None,$37K,$66K (Glassdoor est.)


In [408]:
df['Min_Salary']=df['Min_Salary'].str.strip(' ').str.lstrip('$').str.rstrip('K')

In [409]:
df['Max_Salary']=df['Max_Salary'].str.extract(r'(\d+)')

In [410]:
df['Max_Salary']=df['Max_Salary'].str.strip(' ')

In [411]:
df.drop(['Salary Estimate'],axis=1,inplace=True)

In [412]:

df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [413]:
df.drop(['none'],axis=1,inplace=True)

In [414]:
df.dtypes

Job Title             object
Job Description       object
Rating               float64
Company Name          object
Location              object
Headquarters          object
Size                  object
Founded              float64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
Competitors           object
Easy Apply            object
city                  object
state                 object
Min_Salary            object
Max_Salary            object
dtype: object

In [416]:
df.to_csv('DataAnalyst_clean.csv')